In [2]:
with open("gameofthrones.txt", "r") as file:
    text = file.read()

In [3]:
print("Lenght of the book in characters: ", len(text))

Lenght of the book in characters:  5662324


In [4]:
print(text[:1000])



“We should start back,” Gared urged as the woods began to grow dark around them. “The wildlings are dead.”

“Do the dead frighten you?” Ser Waymar Royce asked with just the hint of a smile.

Gared did not rise to the bait. He was an old man, past fifty, and he had seen the lordlings come and go. “Dead is dead,” he said. “We have no business with the dead.”

“Are they dead?” Royce asked softly. “What proof have we?”

“Will saw them,” Gared said. “If he says they are dead, that’s proof enough for me.”

Will had known they would drag him into the quarrel sooner or later. He wished it had been later rather than sooner. “My mother told me that dead men sing no songs,” he put in.

“My wet nurse said the same thing, Will,” Royce replied. “Never believe anything you hear at a woman’s tit. There are things to be learned even from the dead.” His voice echoed, too loud in the twilit forest.

“We have a long ride before us,” Gared pointed out. “Eight days, maybe nine. And night is falling.”

Ser

Let's explore the influence of the tokenization scheme on the vocabulary size, the context length and the number of tokens in the corpus.

In [19]:
# Character by character encoding
chars = sorted(list(set(text)))
print("Number of unique characters: ", len(chars))

Number of unique characters:  86


In [20]:
# Word by word encoding
words = text.split()
print("Number of unique words: ", len(list(set(words))))

Number of unique words:  54729


In [30]:
# Subword encoding using BPE 
from tokenizers import BertWordPieceTokenizer
tok = BertWordPieceTokenizer()
tok.train(["gameofthrones.txt"], vocab_size=4000, min_frequency=2, special_tokens=[
    "<start>",
    "<end>",
    "<pad>"
])
print("Number of unique subwords: ", len(tok.get_vocab()))

print(tok.encode("The quick brown fox jumps over the lazy dog.").tokens)

Number of unique subwords:  4000
['the', 'quick', 'brown', 'fo', '##x', 'j', '##ump', '##s', 'over', 'the', 'l', '##az', '##y', 'dog', '.']


In [5]:
chars = sorted(list(set(text)))
print(repr("".join(chars)))
print("Number of unique characters: ", len(chars))

'\n !(),-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijklmnopqrstuvwxyz{}éê—‘’“”…'
Number of unique characters:  86


In [6]:
# Create a mapping from characters to indices (vocabulary)
char_to_idx = {ch:i for i, ch in enumerate(chars)}
idx_to_char = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [char_to_idx[c] for c in s] # s is the input string that I want to encode
decode = lambda l: "".join([idx_to_char[i] for i in l]) # l is the input list of indices that I want to decode

In [7]:
# Try to encode and decode a string
string = "Hello World!"
encoded = encode(string)
print("Encoded string: ", encoded)
decoded = decode(encoded)
print("Decoded string: ", decoded)

Encoded string:  [29, 54, 61, 61, 64, 1, 44, 64, 67, 61, 53, 2]
Decoded string:  Hello World!


In [8]:
import torch 
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:100])

torch.Size([5662324])
tensor([ 0,  0, 83, 44, 54,  1, 68, 57, 64, 70, 61, 53,  1, 68, 69, 50, 67, 69,
         1, 51, 50, 52, 60,  5, 84,  1, 28, 50, 67, 54, 53,  1, 70, 67, 56, 54,
        53,  1, 50, 68,  1, 69, 57, 54,  1, 72, 64, 64, 53, 68,  1, 51, 54, 56,
        50, 63,  1, 69, 64,  1, 56, 67, 64, 72,  1, 53, 50, 67, 60,  1, 50, 67,
        64, 70, 63, 53,  1, 69, 57, 54, 62,  7,  1, 83, 41, 57, 54,  1, 72, 58,
        61, 53, 61, 58, 63, 56, 68,  1, 50, 67])


In [9]:
# Split the dataset in training and validation
# I want to store 90% of characters for training and 10% for validation
n = int(0.9*len(text))
train_data = data[:n]
val_data = data[n:]

In [10]:
import random

context_length = 8 # This is maximum number of tokens that are allowed to fit in the context. 

def get_batch(data, batch_size = 8, context_length = 8):
    # Get batch_size random indices in the data
    random_idx = random.sample(range(len(data)-context_length), batch_size)
    # Pluck the next character after each random index
    inputs = torch.zeros((batch_size, context_length), dtype=torch.long)
    targets = torch.zeros((batch_size, context_length), dtype=torch.long)
    
    for i in range(batch_size):
        inputs[i,:] = data[random_idx[i]:random_idx[i]+context_length]
        targets[i,:] = data[random_idx[i]+1:random_idx[i]+context_length+1]
    
    return inputs, targets

In [11]:
# Let's build a bigram language model 
import torch.nn as nn 
import torch.nn.functional as F

class BigramLM(nn.Module):
    def __init__(self, vocab_size):
        super(BigramLM, self).__init__()
        self.embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, inputs):
        # Receive inputs in the form (B, L, C) (batch times seq length times num classes (tokens))
        # Embed the inputs
        embeddings = self.embedding_table(inputs)
        return embeddings
    
    def generate(self, inputs, max_new_tokens=10):
        # Generate next tokens given the inputs (B, L, C)
        for i in range(max_new_tokens):                   
            # Embed the inputs
            logits = self(inputs)
            # Get the last logit 
            last_logit = logits[:, -1, :] # Get the last element in the length dimension
            # Compute the probabilities
            probs = F.softmax(last_logit, dim=1)
            # Get the next token
            next_token = torch.multinomial(probs, num_samples=1)
            # Append to the inputs
            inputs = torch.cat((inputs, next_token), dim=1)
            
        return inputs

In [185]:
model = BigramLM(len(chars))

inputs_generate = torch.zeros((1,1), dtype=torch.long)
print(decode(model.generate(inputs_generate, max_new_tokens=100)[0].tolist()))


/ki!V(Jy—J{!ac!Bc3!ZwdsB07(,W7I.QM3(H8[cRsU2IPo-eU2
éWbfWoNr)nWZLAS.‘}D(5’J1rP
IDF.s5éJvdSPL:!—i

T]


In [186]:
# Let's train this model
import torch.optim as optim
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

# Hyperparameters
learning_rate = 0.01
momentum = 0.9
batch_size = 32
context_length = 8 # Is not0 really taken into consideration here because we are using a bigram model
num_iterations = 100000
device = "cpu"

# Get the optimizer
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
# Get loss function
loss_fn = CrossEntropyLoss()
# Send the model to the device
model.to(device)

loss_train = 0 
# Start training loop
for i in tqdm(range(num_iterations)):
    # Get a batch
    inputs, targets = get_batch(train_data, batch_size=8, context_length=8)
    # Send the inputs and targets to device
    inputs = inputs.to(device)
    targets = targets.to(device)
    # Get the predictions
    predictions = model(inputs)
    # We have to reshape the predictions and the targets to use cross entropy 
    B, L, C = predictions.shape
    predictions = predictions.view(B*L, C)
    targets = targets.view(B*L)
    # Compute the loss
    loss = loss_fn(predictions, targets)
    # Accumulate the loss
    loss_train += loss.item()
    # Zero the gradients
    optimizer.zero_grad()
    # Compute the gradients
    loss.backward()
    # Update the parameters
    optimizer.step()

    if (i%1000)==0 and i!=0:
        print(loss_train/1000)
        loss_train = 0

  1%|          | 1081/100000 [00:01<02:13, 738.40it/s]

4.531553933858872


  2%|▏         | 2119/100000 [00:03<02:24, 677.58it/s]

3.981821083068848


  3%|▎         | 3102/100000 [00:04<02:30, 645.76it/s]

...

 98%|█████████▊| 98128/100000 [03:02<00:02, 710.17it/s]

2.45628770840168


 99%|█████████▉| 99122/100000 [03:03<00:01, 699.51it/s]

2.463740375876427


100%|██████████| 100000/100000 [03:05<00:00, 539.97it/s]


In [190]:
# Let's try to generate
inputs_generate = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(inputs_generate, max_new_tokens=100)[0].tolist()))


“I’te warce thes blimier, gng J‘WCAroshetothe.” Ry iroutore8’s. thergs I{jE—ld, ouse soono oure lo t


Ok, enough for the bigramLM, let's try to switch to some better architecture (that take into consideration the context)

In [191]:
# Let's try with an RNN

class RNNLM(nn.Module):
    def __init__(self, vocab_size, hidden_size=32):
        super(RNNLM, self).__init__()
        self.embedding_table = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        # Classifier to predict the next token
        self.linear = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, inputs):
        # Embed the inputs
        embeddings = self.embedding_table(inputs)
        # Pass the embeddings through the RNN
        outputs, _ = self.rnn(embeddings)
        # Pass the outputs through the classifier
        logits = self.linear(outputs)
        
        return logits

    def generate(self, inputs, max_new_tokens=10):
        # Generate next tokens given the inputs (B, L, C)
        for _ in range(max_new_tokens):
            # Embed the inputs
            embeddings = self.embedding_table(inputs)
            # Feed the RNN with the embeddings 
            _, hidden = self.rnn(embeddings)
            # Remove the first dimension (get only the last hidden state for each element in the batch)
            hidden = hidden.squeeze(0) 
            # Project to the output classes
            logits = self.linear(hidden)
            # Convert to probabilities
            probs = F.softmax(logits, dim=1)
            # Get the next token
            next_token = torch.multinomial(probs, num_samples=1)
            # Concatenate to input
            inputs = torch.cat((inputs, next_token), dim=1)
            
        return inputs

In [130]:
model = RNNLM(len(chars))
# Get a batch 
inputs, targets = get_batch(train_data, batch_size=8, context_length=8)

inputs_generate = torch.zeros((1,1), dtype=torch.long)
print(decode(model.generate(inputs_generate, max_new_tokens=100)[0].tolist()))


wêT…aw};9EShV?e;gJ5-VeXM0DL7?Mj3d16/êê”QX1]DOC5xbU-—!L?SOk—v,,iA2pVRCfIa’ N{x;iuS}êE
QSa)}V ]RkR—““d


In [194]:
# Let's train this model
import torch.optim as optim
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

# Hyperparameters
learning_rate = 0.01
momentum = 0.9
batch_size = 32
context_length = 8 # Here it is important because we are using an RNN
hidden_size = 32
num_iterations = 100000
device = "cpu"

# Create the model 
model = RNNLM(len(chars), hidden_size=hidden_size)
# Send the model to the device
model.to(device)

# Get the optimizer
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
# Get loss function
loss_fn = CrossEntropyLoss()


# Start training loop
loss_train = 0
for i in tqdm(range(num_iterations)):
    # Get a batch
    inputs, targets = get_batch(train_data, batch_size=8, context_length=8)
    # Send the inputs and targets to device
    inputs = inputs.to(device)
    targets = targets.to(device)
    # Get the predictions
    predictions = model(inputs)
    # We have to reshape the predictions and the targets to use cross entropy 
    B, L, C = predictions.shape
    predictions = predictions.view(B*L, C)
    targets = targets.view(B*L)
    # Compute the loss
    loss = loss_fn(predictions, targets)
    # Accumulate the loss
    loss_train += loss.item()
    # Zero the gradients
    optimizer.zero_grad()
    # Compute the gradients
    loss.backward()
    # Update the parameters
    optimizer.step()

    if (i%1000)==0 and i!=0:
        print(loss_train/1000)
        loss_train = 0

  1%|          | 1104/100000 [00:02<02:46, 594.56it/s]

2.697427982211113


  2%|▏         | 2071/100000 [00:03<02:43, 600.52it/s]

2.341876891255379


  3%|▎         | 3094/100000 [00:05<02:40, 603.10it/s]

...

 98%|█████████▊| 98054/100000 [04:09<00:05, 371.28it/s]

2.000637235164642


 99%|█████████▉| 99041/100000 [04:12<00:02, 342.00it/s]

1.9963573273420334


100%|██████████| 100000/100000 [04:14<00:00, 392.49it/s]


In [198]:
inputs_generate = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(inputs_generate, max_new_tokens=100)[0].tolist()))



“The herisses to said, bing tridane say as calmeed. Gord,s to up. HEim besess her hersesserselped o


In [12]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 
# Let's try with an RNN

class GRULM(nn.Module):
    def __init__(self, vocab_size, hidden_size=32):
        super(GRULM, self).__init__()
        self.embedding_table = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.GRU(hidden_size, hidden_size, batch_first=True)
        # Classifier to predict the next token
        self.linear = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, inputs):
        # Embed the inputs
        embeddings = self.embedding_table(inputs)
        # Pass the embeddings through the RNN
        outputs, _ = self.rnn(embeddings)
        # Pass the outputs through the classifier
        logits = self.linear(outputs)
        
        return logits

    def generate(self, inputs, max_new_tokens=10):
        # Generate next tokens given the inputs (B, L, C)
        for _ in range(max_new_tokens):
            # Embed the inputs
            embeddings = self.embedding_table(inputs)
            # Feed the RNN with the embeddings 
            _, hidden = self.rnn(embeddings)
            # Remove the first dimension (get only the last hidden state for each element in the batch)
            hidden = hidden.squeeze(0) 
            # Project to the output classes
            logits = self.linear(hidden)
            # Convert to probabilities
            probs = F.softmax(logits, dim=1)
            # Get the next token
            next_token = torch.multinomial(probs, num_samples=1)
            # Concatenate to input
            inputs = torch.cat((inputs, next_token), dim=1)
            
        return inputs

In [13]:
model = GRULM(len(chars))

inputs_generate = torch.zeros((1,1), dtype=torch.long)
print(decode(model.generate(inputs_generate, max_new_tokens=100)[0].tolist()))


!dxR6X(EjfG9!mD[s-ai3]Vjy1UnopoCM‘muQY),XRM8VP“d:Lzs.n’’(.6C25é}xJ,tO2l.5{;!nE0!VN/.IjbagTC25HrN[xSd


In [16]:
# Let's train this model
import torch.optim as optim
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

# Hyperparameters
learning_rate = 0.01
momentum = 0.9
batch_size = 32
context_length = 8 # Here it is important because we are using an RNN
hidden_size = 32
num_iterations = 100000
device = "cpu"

# Get the model 
model = GRULM(len(chars))
# Send the model to the device
model.to(device)

# Get the optimizer
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
# Get loss function
loss_fn = CrossEntropyLoss()

# Start training loop
loss_train = 0
for i in tqdm(range(num_iterations)):
    # Get a batch
    inputs, targets = get_batch(train_data, batch_size=8, context_length=8)
    # Send the inputs and targets to device
    inputs = inputs.to(device)
    targets = targets.to(device)
    # Get the predictions
    predictions = model(inputs)
    # We have to reshape the predictions and the targets to use cross entropy 
    B, L, C = predictions.shape
    predictions = predictions.view(B*L, C)
    targets = targets.view(B*L)
    # Compute the loss
    loss = loss_fn(predictions, targets)
    # Accumulate the loss
    loss_train += loss.item()
    # Zero the gradients
    optimizer.zero_grad()
    # Compute the gradients
    loss.backward()
    # Update the parameters
    optimizer.step()

    if (i%1000)==0 and i!=0:
        print(loss_train/1000)
        loss_train = 0

  1%|          | 1052/100000 [00:04<06:26, 256.26it/s]

2.843295238018036


  2%|▏         | 1979/100000 [00:08<07:12, 226.38it/s]


KeyboardInterrupt: 

In [207]:
inputs_generate = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(inputs_generate, max_new_tokens=100)[0].tolist()))



“They me of of the trooofre togstered her. A chil?

The’ld the was of they kie her knushing, godge-


In [14]:
# Do the study with different parameters and plot all the results 
# RNN 
import torch.optim as optim
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

# Use pickle to save the loss array
import pickle

# Hyperparameters
learning_rate = 0.01
momentum = 0.9
batch_size = 32
context_length = [512] # Here it is important because we are using an RNN
hidden_size = [256]
num_iterations = 100000
device = "cuda:0"

# Start training loop
for c in context_length:
    for h in hidden_size:
        print("Training with context length: ", c, " and hidden size: ", h)
        # Get the model 
        model = GRULM(len(chars), hidden_size=h)
        # Send the model to the device
        model.to(device)
        # Get the optimizer
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
        # Get loss function
        loss_fn = CrossEntropyLoss()
        
        loss_train = 0
        loss_array = []
        for i in tqdm(range(num_iterations)):
            # Get a batch
            inputs, targets = get_batch(train_data, batch_size=batch_size, context_length=c)
            # Send the inputs and targets to device
            inputs = inputs.to(device)
            targets = targets.to(device)
            # Get the predictions
            predictions = model(inputs)
            # We have to reshape the predictions and the targets to use cross entropy 
            B, L, C = predictions.shape
            predictions = predictions.view(B*L, C)
            targets = targets.view(B*L)
            # Compute the loss
            loss = loss_fn(predictions, targets)
            # Accumulate the loss
            loss_train += loss.item()
            # Zero the gradients
            optimizer.zero_grad()
            # Compute the gradients
            loss.backward()
            # Update the parameters
            optimizer.step()

            if (i%1000)==0 and i!=0:
                loss_array.append(loss_train/1000)
                loss_train = 0

        # Save the loss list 
        with open("loss_rnn_context"+str(c)+"_hiddensize"+str(h)+".pkl", "wb") as file:
            pickle.dump(loss_array, file)

Training with context length:  512  and hidden size:  256


100%|██████████| 100000/100000 [44:05<00:00, 37.80it/s]


In [17]:
inputs_generate = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(inputs_generate, max_new_tokens=1000)[0].tolist()))



“Why would help me to it.” Go rode his helm. Maryon’s happing third Small king was color-things dark suit of the aware old forbug. Ser Boros strode by log twists than Harbel as an silver shield she had selded him at a bejoies. The alehants rushed in his sand’s throat. Bran shook his bone-mooking ovention with her own pair hottering downstrings against you, no matter rock of animals. A Tennmaid the Skirlings, Sansa’s heart where whenever for the varrows, Mucah… the wagon of Clegane’s stone. There is a cumpless come sewn square as the blew back in the truth. The thinner stewled into the get. On a tink of field Lady Mathis pointed displeastriamondy. There would hammered a troth the seal as they listen, that mox and offendage will be all it would not attendly not as us on the stories, and little time summers.” He raved stripping off his endlessoard were screaming beneath him that the treacheroom, there were they were safe into the songs.

“What,” Robb Stark Tallhart. Sansa told herself m

In [18]:
# Save the model
torch.save(model.state_dict(), "model_grurnn_context"+str(c)+"_hiddensize"+str(h)+".pt")